# Simulation de robot à Patte

Cette feuille de travail (Jupyter Notebook) est dédiée à l'expérimentation de mouvement sur un robot à patte quadrupède.

Elle comporte quelques explications et du code python. Les cellules de code sont exécutables et modifiables. Lorqu'on exécute une cellule de code, un interpréteur de code est exécuté en tâche de fond. A noter que l'interpréteur n'est pas relancé à chaque cellule. Pour le relancer, il faut sélectionner dans le menu "Noyau/Redémarrer".

## Préliminaires

En premier lieu il faut charger les bibliothèques qui nous intéresse. `time` et `math` sont classiques. En revanche, `pybullet` l'est moins. C'est le simulateur physique dans lequel va vivre notre robot.

Exécutez la cellule suivante. Pour ce faire, selectionnez-la et cliquez sur le bouton *Exécuter* en dessous du menu.

In [1]:
import math
from time import sleep
import pybullet as p

Dans la cellule suivante, on charge le robot dans le simulateur. C'est une partie plutôt technique, nous n'allons pas nous intérésser.

In [2]:
physicsClient = p.connect(p.GUI)
p.setGravity(0, 0, -9.80665)
planeId = p.loadURDF('plane.urdf')
cubeStartPos = [0, 0, 0.1]
cubeStartOrientation = p.getQuaternionFromEuler([0, 0, 0])
quadruped = p.loadURDF("./quadruped/robot.urdf",
                       cubeStartPos, cubeStartOrientation)
cubePos, cubeOrn = p.getBasePositionAndOrientation(quadruped)
jointsMap = [0, 1, 2, 4, 5, 6, 8, 9, 10, 12, 13, 14]

Si tout va bien, une fenêtre est apparue. C'est la vue du simulateur. On y voit notre robot, mais d'un peu loin. Vous pouvez zoomer avec la molette de la souris.

# Mouvements

Pour le moment, le robot est immobile, et même en lévitation :). Il n'y a pas de simulation physique pour le moment. La simulation physique est une boucle qui à chaque tour calcule le nouvel état des objets qu'elle considère. Sous le mot état, on entend les positions, les vitesses, les forces, etc.

Dans la cellule suivante, on fixe le pas temporel de simulation, ici 10ms. Exécutez-la. Il ne se passera rien, on ne fait que définir le pas pour la simulation future.

In [3]:
dt = 0.01
p.setPhysicsEngineParameter(fixedTimeStep=dt)

Nous allons piloter le robot. Pour ce faire, nous allons à chaque pas de la simulation, envoyer un ordre de position à chaque articulation. On entend par position un angle (donné en radian).  

La fonction suivante a pour but de simuler un mouvement. Ici, un mouvement est une fonction `motion` qui à un temps `t` donné fournit une liste de 12 angles. Ce sont les positions que l'on veut pour les 12 moteurs du robot. La fonction est une boucle temporelle qui à chaque temps (tic) envoie au simulateur un nouvel ordre de position pour chaque moteur.

In [4]:
def simulate(motion, duration):
    t = 0
    while t<duration:
        t += dt
        joints = motion(t) # one gets angular orders for each motor
                           # motion is a function supposed to return a list of 12 angles
        for k in range(len(joints)):
            # here one sends the orders to the simulator for each motor
            jointInfo = p.getJointInfo(quadruped, jointsMap[k])
            p.setJointMotorControl2(quadruped, jointInfo[0], p.POSITION_CONTROL, joints[k])
        # then one launchs the simulation step 
        p.stepSimulation()        
        sleep(dt)

Dans la cellule suivante, on définit la fonction qui va fournir ces ordres angulaires pour chacune des articulations. Concrêtement, c'est une liste de 12 valeurs. Notez qu'il y a 12 moteurs. Pour tester, le mouvement est très simple. Il fait bouger de façon périodique les 3 premiers moteurs du robot.

In [ ]:
def hello_world_motion(t):
    joints = [0]*12
    for joint in [0,1,2]:
        joints[joint] = math.sin(t)
    return joints

Ensuite, on lance la simulation pour 10 secondes:

In [ ]:
#simulate(hello_world_motion, 10)

In [ ]:
# Contient les angles de chaque articulation
joints = [0]*12

# Contient les positions de chaque patte
paws = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]

l1 = 4.5
l2 = 6.5
l3 = 8.7

# Prend les coordonées où on veut poser la patte et modifie les angles pour chaque articulation de la patte
def set_joints(paw, x, y, z):
    teta1 = math.atan2(y, x)
    A = [l1 * math.cos(teta1), l1 * math.sin(teta1)]
    AM = math.sqrt((x - A[0])**2 + (y - A[1])**2 + z**2)
    teta3 = math.acos((l2**2 + l3**2 - AM**2) / (2 * l2 * l3))
    alpha = math.acos((l2**2 - l3**2 + AM**2) / (2 * l2 * AM))
    beta = -math.asin(z / AM)
    teta2 = math.pi - (alpha - beta)
    joints[3 * paw] = teta1
    joints[3 * paw + 1] = math.pi - teta2
    joints[3 * paw + 2] = math.pi - teta3
    
# Met à jour les coordonnées de chaque patte
def set_paws(paw, x, y, z):
    paws[paw][0] = x
    paws[paw][1] = y
    paws[paw][2] = z

# Retourne les coordonnées de la patte
def get_paws(paw):
    return paws[paw]

# Lève la patte paw de delta
def raise_paw(paw, delta):
    p = get_paws(paw)
    set_joints(paw, p[0], p[1], p[2] + delta)
    set_paws(paw, p[0], p[1], p[2] + delta)

# Baisse la patte paw de delta
def drop_paw(paw, delta):
    p = get_paws(paw)
    set_joints(paw, p[0], p[1], p[2] - delta)
    set_paws(paw, p[0], p[1], p[2] - delta)
    
# Décale la patte paw de delta dans la direction x
def shift_pawx(paw, delta):
    p = get_paws(paw)
    set_joints(paw, p[0] + delta, p[1], p[2])
    set_paws(paw, p[0] + delta, p[1], p[2])
    
# Décale la patte paw de delta dans la direction y
def shift_pawy(paw, delta):
    p = get_paws(paw)
    set_joints(paw, p[0], p[1] + delta, p[2])
    set_paws(paw, p[0], p[1] + delta, p[2])

A vous de jouer ! Pour commencer, faite un mouvement où le robot reste immobile, mais les pattes posées, la seconde articulation faisant un angle droit. 
  
La fonction immobile décrit le mouvement. C'est à dire que pour chaque `t`, elle doit fournir les angles de chaque moteur sous la forme d'une liste de 12 valeurs réelles (des floats). De la même façon que le faisait la fonction `hello_world_motion` précédente. Dans le cas d'`immobile`, vous devez fournir les mêmes valeurs quelque soit la valeur de `t`.

In [ ]:
def immobile(t):
    for i in range(4):
        set_joints(i, l1 + l2, 0, -l3)
        set_paws(i, l1 + l2, 0, -l3)
    return joints      

Comme précédemment, définir la fonction `immobile` n'est pas suffisant, il faut lancer le simulateur (ici pour 5 secondes):

In [ ]:
simulate(immobile, 3)

En faisant appel à votre imagination, créez un premier mouvement de déplacement.  
  
Vous allez voir rapidement que cela n'est pas immédiat. Pour cette première approche vous pouvez appliquer la stratégie suivante:
- considérer une succession arbitraire d'intervalles de temps $I_k$ partitionnant une durée finie qui sera la période $p$ du mouvement. Période que vous fixerez également de façon arbitraire.
- pour chacun de ces intervalles $I_k$, definissez une position $P_k$ du robot (comme précédemment dans `immobile`). Cette position est arbitraire, c'est à vous de la définir pour créer votre mouvement.

La fonction `deplacement` ainsi va:
- déterminer l'intervalle $I_k$ auquel appartient `t`, où plus exactement, `t` "ramené" à la période $p$ (utilisez la fonction `math.fmod(t,p)`)
- puis retourner la position $P_k$ correspondante.


In [ ]:
# Modifier la vitesse v entraine que le robot se déplace d'un delta plus grand
v = 5
# Temps que l'on passe dans chaque if de la fonction deplacement
cycle = 500 * dt / v
# Nombre d'états du robot soit le nombre de if dans déplacement
nombre_d_etats = 7

# Si lateral est mis à 1 alors on se deplace sur les cotés (gauche et droite) s'il est mis à 0 on se déplace en avant ou en arrière
lateral = 1
# Deltas qui régissent le sens de direction
delta_raise = 0.03 * v 
delta_shift = 0.03 * v # Mettre un plus ou un moins devant delta_shift pour le faire deplacer vers la gauche ou l'arriere en fonction de la valeur de lateral

def deplacement(t):
    global delta, cycle, nombre_d_etats, lateral, cote
    
    if ((math.fmod(t, cycle) <= cycle / nombre_d_etats)):
        immobile(t)
        
    elif ((math.fmod(t, cycle) <= 2 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > cycle / nombre_d_etats)):
        if (lateral == 1):
            raise_paw(1, delta_raise)
            raise_paw(3, delta_raise)
        else:
            raise_paw(0, delta_raise)
            raise_paw(2, delta_raise)
        
    elif ((math.fmod(t, cycle) <= 3 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > 2 * cycle / nombre_d_etats)):
        if (lateral == 1):
            shift_pawx(1, -delta_shift)
            shift_pawx(3, delta_shift)
            shift_pawy(0, delta_shift)
            shift_pawy(2, -delta_shift)
        else:
            shift_pawx(0, delta_shift)
            shift_pawx(2, -delta_shift)
            shift_pawy(1, delta_shift)
            shift_pawy(3, -delta_shift)
        
    elif ((math.fmod(t, cycle) <= 4 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > 3 * cycle / nombre_d_etats)):
        if (lateral == 1):
            drop_paw(1, delta_raise)
            drop_paw(3, delta_raise)
        else:
            drop_paw(0, delta_raise)
            drop_paw(2, delta_raise)
        
    elif ((math.fmod(t, cycle) <= 5 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > 4 * cycle / nombre_d_etats)):
        if (lateral == 1):
            raise_paw(0, delta_raise)
            raise_paw(2, delta_raise)
        else:
            raise_paw(1, delta_raise)
            raise_paw(3, delta_raise)
        
    elif ((math.fmod(t, cycle) <= 6 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > 5 * cycle / nombre_d_etats)):
        if (lateral == 1):
            shift_pawy(0, -delta_shift)
            shift_pawy(2, delta_shift)
            shift_pawx(1, delta_shift)
            shift_pawx(3, -delta_shift)
        else:
            pass
            shift_pawy(1, -delta_shift)
            shift_pawy(3, delta_shift)
            shift_pawx(0, -delta_shift)
            shift_pawx(2, delta_shift)

    elif ((math.fmod(t, cycle) <= 7 * cycle / nombre_d_etats) and (math.fmod(t, cycle) > 6 * cycle / nombre_d_etats)):
        if (lateral == 1):
            drop_paw(0, delta_raise)
            drop_paw(2, delta_raise)
        else:
            drop_paw(1, delta_raise)
            drop_paw(3, delta_raise)
     
    return joints

In [ ]:
simulate(deplacement, 1000)